In [428]:
import pandas as pd

from ib_insync import *

# from  configuration.ibtrader_functions import *
from  configuration.ibtrader_settings import * 
from  configuration.ibtrader_stocks import * 
import matplotlib.dates as mpl_dates
import pandas_ta as ta
import yfinance

In [430]:
#name = 'ENG.MC'
#ticker = yfinance.Ticker(name)
#dfDaily = ticker.history(interval="1d",start="2015-07-01", end="2015-08-30")
dfDaily = pd.read_csv("C:\\Users\\DAVIDNEVADO\\git\\py_ibtrader\\data\\DAY\\TSLA_2020_08_13.csv")

In [431]:
dfDaily

,Unnamed: 0,date,open,high,low,close,volume,average,barCount
0,0,2015-08-17,255.56,256.59,250.51,254.99,56479,253.3425,33782
1,1,2015-08-18,255.38,260.95,253.56,260.72,35083,257.8435,21843
2,2,2015-08-19,260.33,260.65,255.02,255.25,30510,257.4445,17799
3,3,2015-08-20,252.06,254.56,241.90,242.18,40930,246.7425,25990
4,4,2015-08-21,236.00,243.80,230.51,230.77,54384,235.7405,34078
...,...,...,...,...,...,...,...,...,...
1253,1253,2020-08-07,1500.00,1500.00,1415.01,1452.71,46369,1452.0975,27475
1254,1254,2020-08-10,1448.00,1457.50,1385.84,1418.57,38165,1420.1385,22313
1255,1255,2020-08-11,1395.90,1420.00,1364.99,1374.39,37821,1394.6115,21128
1256,1256,2020-08-12,1470.00,1585.00,1435.00,1554.76,111002,1517.0630,68659


In [432]:
dfDaily['date'] = dfDaily['Date']
dfDaily['open'] = dfDaily['Open']
dfDaily['high'] = dfDaily['High']
dfDaily['low'] = dfDaily['Low']
dfDaily['close'] = dfDaily['Close']


dfDaily = dfDaily.loc[:,['date', 'open', 'high', 'low', 'close']]


KeyError: 'Date'

In [433]:
dfDaily

,Unnamed: 0,date,open,high,low,close,volume,average,barCount
0,0,2015-08-17,255.56,256.59,250.51,254.99,56479,253.3425,33782
1,1,2015-08-18,255.38,260.95,253.56,260.72,35083,257.8435,21843
2,2,2015-08-19,260.33,260.65,255.02,255.25,30510,257.4445,17799
3,3,2015-08-20,252.06,254.56,241.90,242.18,40930,246.7425,25990
4,4,2015-08-21,236.00,243.80,230.51,230.77,54384,235.7405,34078
...,...,...,...,...,...,...,...,...,...
1253,1253,2020-08-07,1500.00,1500.00,1415.01,1452.71,46369,1452.0975,27475
1254,1254,2020-08-10,1448.00,1457.50,1385.84,1418.57,38165,1420.1385,22313
1255,1255,2020-08-11,1395.90,1420.00,1364.99,1374.39,37821,1394.6115,21128
1256,1256,2020-08-12,1470.00,1585.00,1435.00,1554.76,111002,1517.0630,68659


In [434]:
def isSupport(df,i,keyField):
  support = (df[keyField][i] < df[keyField][i-1]  and df[keyField][i] < df[keyField][i+1]  and df[keyField][i+1] < df[keyField][i+2] and df[keyField][i-1] < df[keyField][i-2])
  #print (str(df[keyField][i-1]) + "-" + str(df[keyField][i]) + "-" + str(df[keyField][i+1]))
  #print ("Support?:" + str(i) + "-" + str(support))
  return support
def isResistance(df,i,keyField):
  resistance = df[keyField][i] > df[keyField][i-1]  and df[keyField][i] > df[keyField][i+1]  and df[keyField][i+1] > df[keyField][i+2] and df[keyField][i-1] > df[keyField][i-2]
  return resistance
def isFarFromLevel(l,levels, mean):
  return np.sum([abs(l-x) < mean  for x in levels]) == 0

In [445]:
# TAMBIEN VALE PARA VALLES Y PICOS DE INDICADORES 
def getSupportResistances(dfData):

    support_levels = []
    resistance_levels  = []
    mean_value =  np.mean(dfData['high'] - dfData['low'])
    for i in range(2,dfData.shape[0]-2):
        # Finally, let’s create a list that will contain the levels we find. Each level is a tuple whose first element is the index of the signal candle and the second element is the price value.
        # print (support_levels)
        if isSupport(dfData,i,"close"):
            l = dfData['close'][i]
            #print (isFarFromLevel(l,support_levels,mean_value))
            # if isFarFromLevel(l,support_levels,mean_value):            
            support_levels.append((i,l))
            #print (support_levels)
        elif isResistance(dfData,i,"close"):
            l = dfData['close'][i]
            #if isFarFromLevel(l,resistance_levels,mean_value):
            resistance_levels.append((i,l))        
    return resistance_levels,support_levels

In [446]:
# TAMBIEN VALE PARA VALLES Y PICOS DE INDICADORES 
def getPeaksValleys(dfData,keyField):
    valleys_levels = []
    peaks_levels   = []    
    for i in range(2,dfData.shape[0]-2):
        # Finally, let’s create a list that will contain the levels we find. Each level is a tuple whose first element is the index of the signal candle and the second element is the price value.
        if isSupport(dfData,i,keyField):
            l = dfData[keyField][i]               
            valleys_levels.append((i,l))                 
        elif isResistance(dfData,i,keyField):
            l = dfData[keyField][i]            
            peaks_levels.append((i,l))        
    return peaks_levels,valleys_levels 

In [447]:
dfDaily_original = dfDaily.copy()

resistance_levels_daily, support_levels_daily = getSupportResistances(dfDaily)



In [504]:
dfsma_daily =  dfDaily.ta.sma(length=200, append=True)
macddf_daily = dfDaily.ta.macd(fast=12, slow=26, signal=9, min_periods=None, append=True)
rsidf_daily =  dfDaily.ta.rsi(length=14, append=True)
#efidf =  dfDaily.ta.efi(length=2, append=True)
#print ("Sma daily:for :"  + contract.symbol    + " " +  str(dfsma_daily[dfsma_daily.size-1]))  
#print ("macddf daily:"+ str( macddf_daily[-1:]["MACD_12_26_9"])  )
#print ("rsidf daily:"+ str( rsidf_daily[rsidf_daily.size-1]))  

daily_sma = dfsma_daily[dfsma_daily.size-1]
daily_macd_value = macddf_daily[-1:]["MACD_12_26_9"] # AQUI SACAMOS EL MACD , DIVERGENCIA POR HISTOGRAMA
daily_rsi_value = rsidf_daily[rsidf_daily.size-1]

KeyError: -1

In [505]:
daily_macd_value

1257    56.311257
Name: MACD_12_26_9, dtype: float64

In [506]:
macddf_daily.MACD_12_26_9.iat[-1]  

56.31125716684096

In [500]:
daily_rsi_value

63.926930144093824

In [498]:
daily_macd_value.MACD_12_26_9.iat[-1]  

AttributeError: 'Series' object has no attribute 'MACD_12_26_9'

In [494]:
macddf_daily[macddf_daily.size-1] # AQUI SACAMOS EL MACD , DIVERGENCIA POR HISTOGRAMA


KeyError: 3773

In [450]:
# Fechas soporte
print ("Fechas de sportes")
dfSupportsDate = pd.DataFrame(support_levels_daily,columns=['TimeFrameImdex','Price'])
# SOLO HASTA EL VALOR N-1
for i in range(len(dfSupportsDate)):
    print (dfDaily_original.iloc[support_levels_daily[i][0]]["date"])
        

Fechas de sportes
2015-08-24
2015-09-04
2015-11-23
2016-01-20
2016-01-27
2016-03-23
2016-03-30
2016-06-02
2016-06-14
2016-06-24
2016-07-27
2016-08-17
2016-10-17
2016-11-03
2016-11-21
2016-12-09
2016-12-30
2017-02-16
2017-03-10
2017-04-27
2017-05-23
2017-06-02
2017-06-19
2017-07-06
2017-08-01
2017-08-10
2017-08-21
2017-08-28
2017-09-28
2017-10-16
2018-01-04
2018-01-25
2018-02-05
2018-02-09
2018-02-21
2018-04-17
2018-06-25
2018-07-06
2018-07-30
2018-08-09
2018-08-17
2018-10-11
2018-10-19
2018-11-06
2018-11-12
2019-01-03
2019-01-23
2019-02-21
2019-03-06
2019-03-19
2019-03-25
2019-05-01
2019-06-03
2019-06-26
2019-07-26
2019-08-15
2019-10-04
2019-12-05
2019-12-30
2020-01-10
2020-01-17
2020-01-27
2020-02-28
2020-04-02
2020-05-01
2020-07-31
2020-08-11


In [451]:
resistance_levels_daily

[(21, 262.25),
 (27, 263.12),
 (44, 228.1),
 (58, 232.36),
 (67, 221.8),
 (85, 234.51),
 (109, 202.55),
 (115, 196.94),
 (126, 168.68),
 (149, 238.32),
 (153, 230.26),
 (160, 265.42),
 (204, 235.52),
 (226, 224.78),
 (236, 230.01),
 (240, 234.79),
 (285, 213.7),
 (300, 202.76),
 (311, 194.94),
 (318, 188.66),
 (323, 196.65),
 (345, 219.74),
 (352, 231.28),
 (362, 254.61),
 (377, 280.98),
 (381, 277.39),
 (389, 251.57),
 (415, 312.39),
 (425, 313.79),
 (429, 322.83),
 (450, 341.01),
 (460, 380.66),
 (467, 383.45),
 (510, 352.93),
 (515, 355.9),
 (526, 385.0),
 (548, 359.65),
 (586, 341.03),
 (613, 352.79),
 (624, 345.0),
 (631, 335.49),
 (636, 357.42),
 (663, 305.72),
 (673, 300.08),
 (722, 349.93),
 (729, 322.47),
 (735, 323.85),
 (746, 349.54),
 (753, 356.41),
 (759, 321.9),
 (784, 309.58),
 (798, 276.59),
 (811, 346.41),
 (815, 351.4),
 (821, 354.31),
 (861, 347.31),
 (873, 321.35),
 (889, 319.88),
 (899, 289.96),
 (904, 274.02),
 (935, 255.34),
 (969, 223.64),
 (990, 264.88),
 (994,

In [452]:
# Fechas resistencias
print ("Fechas de resistencias")
dfresistenciassDate = pd.DataFrame(resistance_levels_daily,columns=['TimeFrameImdex','Price'])
# SOLO HASTA EL VALOR N-1
for i in range(len(dfresistenciassDate)):
       print (dfDaily_original.iloc[resistance_levels_daily[i][0]]["date"])


Fechas de resistencias
2015-09-16
2015-09-24
2015-10-19
2015-11-06
2015-11-19
2015-12-16
2016-01-22
2016-02-01
2016-02-17
2016-03-21
2016-03-28
2016-04-06
2016-06-08
2016-07-11
2016-07-25
2016-07-29
2016-10-03
2016-10-24
2016-11-08
2016-11-17
2016-11-25
2016-12-28
2017-01-09
2017-01-24
2017-02-14
2017-02-21
2017-03-03
2017-04-10
2017-04-25
2017-05-01
2017-05-31
2017-06-14
2017-06-23
2017-08-24
2017-08-31
2017-09-18
2017-10-18
2017-12-12
2018-01-23
2018-02-07
2018-02-16
2018-02-26
2018-04-05
2018-04-19
2018-06-28
2018-07-10
2018-07-18
2018-08-02
2018-08-13
2018-08-21
2018-09-26
2018-10-16
2018-11-02
2018-11-08
2018-11-16
2019-01-17
2019-02-05
2019-02-28
2019-03-14
2019-03-21
2019-05-06
2019-06-24
2019-07-24
2019-07-30
2019-08-08
2019-08-19
2019-08-30
2019-09-11
2019-09-26
2019-10-17
2019-10-25
2019-12-03
2019-12-26
2020-01-08
2020-01-14
2020-01-23
2020-03-04
2020-03-25
2020-04-17
2020-06-18


In [464]:
# DAILY 
dfsma_daily =  dfDaily.ta.sma(length=50, append=True)
macddf_daily = dfDaily.ta.macd(fast=8, slow=21, signal=9, min_periods=None, append=True)
rsidf_daily =  dfDaily.ta.rsi(length=14, append=True)
#efidf =  dfDaily.ta.efi(length=2, append=True)
print ("Sma daily:" + str(dfsma_daily[dfsma_daily.size-1]))  
print ("macddf daily:"+ str( macddf_daily[-1:]["MACD_8_21_9"])  )
print ("rsidf daily:"+ str( rsidf_daily[rsidf_daily.size-1]))  
peak_levels_macd,valley_levels_macd = getPeaksValleys(macddf_daily,"MACD_8_21_9")

Sma daily:1280.7019999999995
macddf daily:1257    42.359639
Name: MACD_8_21_9, dtype: float64
rsidf daily:63.926930144093824


In [488]:
macddf_daily

,MACD_8_21_9,MACDH_8_21_9,MACDS_8_21_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1253,35.137302,-29.225725,64.363027
1254,24.064830,-32.238557,56.303388
1255,9.948238,-37.084120,47.032358
1256,23.450784,-18.865259,42.316043


In [466]:

listInxDivergenceUpperMACD = getIndexUpperDivergence(support_levels_daily,valley_levels_macd)
listInxDivergenceLowerMACD = getIndexLowerDivergence(resistance_levels_daily,peak_levels_macd)




print ("Divergences Upper Found:")
print (listInxDivergenceUpperMACD)     
print ("Divergences Lower Found:")
print (listInxDivergenceLowerMACD)     

























Divergences Upper Found:
[]
Divergences Lower Found:
[]


In [454]:
peak_levels_macd

[(24, 7.90147645655685),
 (44, -8.179980125464652),
 (59, 0.46990384272262986),
 (76, 4.568731295331673),
 (88, 2.2440706956716383),
 (95, 3.887049869342235),
 (115, -8.612017313456363),
 (150, 15.86306971481062),
 (161, 16.10201144121075),
 (199, -1.0337734553824305),
 (205, 2.781121917743235),
 (213, -1.297242592457195),
 (233, 4.221504911203482),
 (240, 4.9492782072863974),
 (287, 0.5909224894723764),
 (303, -0.35019802811004297),
 (311, -3.3659840870377877),
 (324, -0.04172859488446079),
 (345, 8.562013758661891),
 (352, 10.283103971211688),
 (363, 12.88524956483866),
 (378, 13.461991150370864),
 (400, 1.556136481744261),
 (416, 16.03255247636787),
 (429, 10.192715664660682),
 (438, 7.992389344452647),
 (456, 16.40268834060737),
 (460, 18.65973832178912),
 (491, -1.4768079693010918),
 (503, 8.756796887727376),
 (510, 1.7123646407484898),
 (516, 2.0463247056709974),
 (526, 10.11575580876081),
 (548, 0.8579722270755497),
 (576, -2.177651358418757),
 (590, 8.799281665012984),
 (613, 9

In [485]:
# RSI series 
dfRSI = pd.DataFrame(rsidf_daily,columns=['RSI_14'])

peak_levels_rsi,valley_levelsrsi = getPeaksValleys(dfRSI,"RSI_14")

listInxDivergenceUpperRSI = getIndexUpperDivergence(support_levels_daily,valley_levelsrsi)
listInxDivergenceLowerRSI = getIndexLowerDivergence(resistance_levels_daily,peak_levels_rsi)

if len(listInxDivergenceUpperRSI)>0:
    print ("Daily listInxDivergenceUpperRSI"  + contract.symbol    + " " )
    # Fechas peak_levels_macd       
    df_rsiDate = pd.DataFrame(listInxDivergenceUpperRSI,columns=['TimeFrameImdex'])
    # SOLO HASTA EL VALOR N-1
    for i in range(len(df_rsiDate)):
        print (dfDaily_original.iloc[listInxDivergenceUpperRSI['TimeFrameImdex'][i]]["date"])            
if len(listInxDivergenceLowerRSI)>0:
    print ("Daily listInxDivergenceLowerRSI"  + contract.symbol    + " " )
    df_rsiDate = pd.DataFrame(listInxDivergenceLowerRSI,columns=['TimeFrameImdex'])    
    # SOLO HASTA EL VALOR N-1
    for i in range(len(df_rsiDate)):        
        print (dfDaily_original.iloc[df_rsiDate['TimeFrameImdex'][i]]["date"])    
















































































Daily listInxDivergenceLowerRSINVDA 
2020-04-17
2020-06-18


In [480]:
df_rsiDate

,TimeFrameImdex
0,1175
1,1218


In [455]:
# Fechas peak_levels_macd
print ("Fechas peak_levels_macd")
dfpeak_levels_macdDate = pd.DataFrame(peak_levels_macd,columns=['TimeFrameImdex','Price'])
# SOLO HASTA EL VALOR N-1
for i in range(len(dfpeak_levels_macdDate)):
       print (dfDaily_original.iloc[peak_levels_macd[i][0]]["date"])


Fechas peak_levels_macd
2015-09-21
2015-10-19
2015-11-09
2015-12-03
2015-12-21
2015-12-31
2016-02-01
2016-03-22
2016-04-07
2016-06-01
2016-06-09
2016-06-21
2016-07-20
2016-07-29
2016-10-05
2016-10-27
2016-11-08
2016-11-28
2016-12-28
2017-01-09
2017-01-25
2017-02-15
2017-03-20
2017-04-11
2017-05-01
2017-05-12
2017-06-08
2017-06-14
2017-07-28
2017-08-15
2017-08-24
2017-09-01
2017-09-18
2017-10-18
2017-11-28
2017-12-18
2018-01-23
2018-02-27
2018-03-13
2018-04-13
2018-04-19
2018-05-10
2018-07-19
2018-07-26
2018-08-08
2018-10-02
2018-11-02
2018-11-08
2018-12-13
2018-12-31
2019-01-17
2019-02-06
2019-02-12
2019-02-28
2019-03-14
2019-03-21
2019-04-03
2019-04-18
2019-05-08
2019-06-24
2019-07-05
2019-07-24
2019-10-02
2019-10-17
2019-10-30
2019-11-12
2019-12-27
2020-01-14
2020-01-23
2020-02-04
2020-02-20
2020-05-11
2020-07-15


In [487]:
diverg_date1 = dfDaily_original["date"][0]
diverg_date1

'2015-08-17'

In [456]:
# Fechas peak_levels_macd
print ("Fechas valley_levels_macd")
dfvalley_levels_macddDate = pd.DataFrame(valley_levels_macd,columns=['TimeFrameImdex','Price'])
# SOLO HASTA EL VALOR N-1
for i in range(len(dfvalley_levels_macddDate)):
           print (dfDaily_original.iloc[valley_levels_macd[i][0]]["date"])

Fechas valley_levels_macd
2015-10-14
2015-10-23
2015-11-16
2015-12-15
2015-12-28
2016-01-28
2016-02-10
2016-03-31
2016-05-12
2016-06-03
2016-06-17
2016-06-27
2016-07-22
2016-09-09
2016-10-17
2016-11-04
2016-11-16
2016-12-02
2017-01-03
2017-03-10
2017-03-23
2017-04-27
2017-05-08
2017-05-24
2017-06-12
2017-07-10
2017-08-02
2017-08-22
2017-08-29
2017-10-02
2017-11-06
2017-12-05
2018-01-04
2018-02-12
2018-03-09
2018-04-02
2018-04-17
2018-04-25
2018-05-24
2018-07-24
2018-07-30
2018-09-07
2018-10-11
2018-11-06
2019-01-03
2019-01-28
2019-02-08
2019-02-22
2019-03-07
2019-03-19
2019-03-25
2019-04-15
2019-05-01
2019-05-28
2019-06-26
2019-07-09
2019-08-07
2019-08-16
2019-08-27
2019-09-25
2019-10-07
2019-10-23
2019-11-05
2019-12-06
2020-01-02
2020-01-17
2020-01-28
2020-02-14
2020-03-19
2020-05-07
2020-05-28
2020-06-26
2020-08-11


In [ ]:
diverg_date1 = dfData.iloc[valley_levels_macd[i][0]]["date"])

In [457]:
dfPeaksPrice = pd.DataFrame(resistance_levels_daily,columns=['TimeFrameImdex','Price'])
dfPeaksIndicator = pd.DataFrame(peak_levels_macd,columns=['TimeFrameImdex','Price'])

dfValleysPrice = pd.DataFrame(support_levels_daily,columns=['TimeFrameImdex','Price'])
dfValleysIndicator = pd.DataFrame(valley_levels_macd,columns=['TimeFrameImdex','Price'])

In [458]:
#print (peak_levels_macd)
#print (valley_levels_macd)

print ("Sma daily:" + str(dfsma[dfsma.size-1]))  
print ("macddf daily:"+ str( macddf[-1:]["MACD_8_21_9"])  )
print ("rsidf daily:"+ str( rsidf[rsidf.size-1]))  
#print ("efidf daily:" + str(efidf[dfsma.size-1])) 
# 

,TimeFrameImdex,Price
0,21,262.25
1,27,263.12
2,44,228.10
3,58,232.36
4,67,221.80
...,...,...
75,1116,572.20
76,1144,749.50
77,1159,539.25
78,1175,753.89


In [459]:
dfPeaksIndicator

,TimeFrameImdex,Price
0,24,7.901476
1,44,-8.179980
2,59,0.469904
3,76,4.568731
4,88,2.244071
...,...,...
68,1116,48.315766
69,1124,102.203264
70,1135,97.180632
71,1191,48.401521


In [460]:
dfValleysPrice

,TimeFrameImdex,Price
0,5,218.87
1,14,241.93
2,69,217.75
3,107,198.70
4,112,188.07
...,...,...
62,1141,667.99
63,1165,454.47
64,1185,701.32
65,1248,1430.76


In [461]:
dfValleysIndicator

,TimeFrameImdex,Price
0,41,-11.011052
1,48,-10.510475
2,64,-3.160173
3,84,-0.893773
4,92,1.700085
...,...,...
68,1155,-107.740952
69,1189,45.001445
70,1203,24.482972
71,1224,33.022255


In [463]:
#print (peak_levels_macd)
#print (valley_levels_macd)

listInxDivergenceUpperMACD = getIndexUpperDivergence(support_levels_daily,valley_levels_macd)
listInxDivergenceLowerMACD = getIndexLowerDivergence(resistance_levels_daily,peak_levels_macd)




print ("Divergences Upper Found:")
print (listInxDivergenceUpperMACD)     
print ("Divergences Lower Found:")
print (listInxDivergenceLowerMACD)     
print ("Sma daily:" + str(dfsma[dfsma.size-1]))  
print ("macddf daily:"+ str( macddf[-1:]["MACD_8_21_9"])  )
print ("rsidf daily:"+ str( rsidf[rsidf.size-1]))  
#print ("efidf daily:" + str(efidf[dfsma.size-1])) 
# 

























Divergences Upper Found:
[]
Divergences Lower Found:
[]
Sma daily:nan


KeyError: 'MACD_8_21_9'

In [5]:
macddf

,MACD_8_21_9,MACDH_8_21_9,MACDS_8_21_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1253,17.118912,2.915015,14.203897
1254,16.409953,1.764845,14.645108
1255,13.922609,-0.577999,14.500608
1256,14.982919,0.385849,14.597071
